## test-google-news-index

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/apple/WRK/dcolinmorgan/dots'

In [2]:
from DOTS.scrape import get_OS_data, get_test_gnews
from DOTS.pull import pull_data, process_url
from DOTS.ingestion_utils import safe_iter_pull, iter_pull, reduce_newlines, scrape_selenium_headless

In [3]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [4]:
from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
os_url = os.getenv('OS_TOKEN')
openai_KEY = os.getenv('OPENAI_API_KEY')

lobstr_key = os.getenv('LOBSTR_KEY')

In [5]:
response = get_test_gnews(1000)
hits = response["hits"]["hits"]
articles = pull_data(hits)
indices = [i for i, x in enumerate(articles) if len(str(x)) < 50]
print(len(indices))
# for i in indices:
#     url = hits[i]['_source']['metadata']['link']
#     # print(url)
#     try:
#         articles[i] = scrape_selenium_headless(url,browser='undetected_chrome')
#     except:
#         pass
# indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
# print(len(indices2))

grabbing text from url: 100%|██████████| 821/821 [02:29<00:00,  5.49it/s]

199


In [6]:
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
print(len(indices2))
articles_not_in_indices2 = [article for i, article in enumerate(articles) if i not in indices2]

199


## GLiNER

In [148]:
from gliner import GLiNER
model = GLiNER.from_pretrained("urchade/gliner_base")

indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
print(len(indices2))
text = [article for i, article in enumerate(articles) if i not in indices2]
hh = [hit for i, hit in enumerate(hits) if i not in indices2]

199


In [181]:
[len(hh),len(text)]

[622, 622]

In [153]:
import pandas as pd

labels=['disaster', 'flood', 'weather', 'hurricane', 'fire', 'date', 'location']#, 'politician', 'corporation', 'government_agency', 'military']
df = pd.DataFrame(columns=['Title','URL', 'Text', 'Label', 'Date','Location'])

for hit,article in zip(hh[:100],text[:100]):
    entities = model.predict_entities(''.join(article), labels)
    for entity in entities:
        row = pd.DataFrame({'Title': [hit['_source']['metadata']['title']],
                            'URL': [hit['_source']['metadata']['link']], 
                            'Text': [entity["text"]], 
                            'Label': [entity["label"]]})
        df = pd.concat([df, row], ignore_index=True)
print(len(df))

1464


In [155]:
df2 = df.groupby(['Title','URL','Label']).agg({'Text': ' '.join}).reset_index()

In [ ]:
df_pivot = df2.pivot(index=['Title','URL'], columns='Label', values='Text')
df_pivot.reset_index(inplace=True)
df_pivot = df_pivot.dropna(subset=['disaster','fire','flood','weather'],how='all')
df_pivot

In [182]:

from collections import Counter

# Your list
lst = df.Text[df.Label=='disaster']

# Count the frequency of each element
counter = Counter(lst)

# Sort the list by frequency
sorted_lst = sorted(lst, key=lambda x: -counter[x])

print(pd.unique(sorted_lst))

['landslide' 'World War Two' 'gas leak' 'famine' 'snow event' 'flooding']


In [185]:

from collections import Counter

# Your list
lst = df.Text[df.Label=='weather']

# Count the frequency of each element
counter = Counter(lst)

# Sort the list by frequency
sorted_lst = sorted(lst, key=lambda x: -counter[x])

print(pd.unique(sorted_lst))

['snow' 'rain' 'tornadoes' 'showers' 'sunshine' 'wind' 'sleet' 'storm'
 'jet stream' 'winds' 'severe weather' 'straight-line winds' 'tornado'
 'wet snow' 'power outages' 'Wind gusts' 'wind gusts' 'gusty winds'
 'Polar air' 'storms' 'Rain' 'floodwaters' 'snow bomb' 'rainfall'
 'snowfall' 'flood waters' 'solar eclipse' 'tornado watches' 'heavy rain'
 'wet flakes' 'large hail' 'severe thunderstorms' 'cold front'
 '60 mph will be the primary concern. However a tornado'
 'tornado warning\xa0means residents should take shelter immediately because a tornado'
 'Tornadoes' 'EF2 tornado' 'straight line winds' 'EF-0 tornado' 'ENE wind'
 'wintry mix' 'Tropical air' 'Atlantic weather systems' 'sub-tropical air'
 'Jet stream' 'water tables' 'tropical air' 'flood water'
 'Wind gusts of 60-70+ mph will be likely. Tree damage' 'Wind Advisories'
 'rain showers' 'lightning' 'flooding' 'thunderstorms' 'hail'
 'Spring sunshine' 'wintry weather' 'Spring sunshineRead' 'temperatures'
 'windy' 'mild' 'Showers'

In [183]:

from collections import Counter

# Your list
lst = df.Text[df.Label=='fire']

# Count the frequency of each element
counter = Counter(lst)

# Sort the list by frequency
sorted_lst = sorted(lst, key=lambda x: -counter[x])

print(pd.unique(sorted_lst))

['fire' 'six-alarm fire' 'Israeli fire' 'Fire' 'truck fire'
 'multi-alarm fire' 'brush fire']


In [31]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key
from graphistry.features import search_model, topic_model, ngrams_model, ModelDict, default_featurize_parameters, default_umap_parameters

In [178]:
%%time
g=graphistry.nodes(df_pivot.drop(['URL','Title','date'],axis=1))
g2 = g.umap()#df_pivot.drop(['URL','Title'],axis=1),**topic_model)
g2 = g2.dbscan() #min_dist=1, min_samples=3)
# g3 = g2.transform_dbscan(df_pivot.drop(['URL','Title'],axis=1),return_graph=False)
df2=pd.DataFrame(g2.get_matrix())

max_index_per_row = df2.idxmax(axis=1)
top_3_indices = max_index_per_row.value_counts().index[:10]
top_3_indices

* Ignoring target column of shape (38, 0) in UMAP fit, as it is not one dimensional

CPU times: user 1.07 s, sys: 240 ms, total: 1.31 s
Wall time: 1.15 s


Index(['disaster_nan', 'disaster_landslide flooding landslide landslide',
       'disaster_famine', 'disaster_snow event', 'disaster_World War Two',
       'disaster_gas leak'],
      dtype='object')

In [179]:
import seaborn as sns
import matplotlib.colors as mcolors
palette = sns.color_palette("hls", 10)
hex_palette = [mcolors.rgb2hex(color) for color in palette]

In [180]:
g2.plot() # encode_point_color('Label', palette=hex_palette, as_continuous=False).plot()

In [9]:
stopppppp

NameError: name 'stopppppp' is not defined

## g.feat

In [ ]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key
from graphistry.features import search_model, topic_model, ngrams_model, ModelDict, default_featurize_parameters, default_umap_parameters

In [ ]:
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
(len(indices2))
articles_not_in_indices2 = [article for i, article in enumerate(articles) if i not in indices2]

df=pd.Series(articles_not_in_indices2).to_frame()
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)
df.columns=['text']
df['text'] = df['text'].str.replace("', '", ' ')
df['text'] = df['text'].str.replace('", "', ' ')
df['text'] = df['text'].str.replace('" "', ' ')
df['text'] = df['text'].str.replace("' '", ' ')
df['text'] = df['text'].str.replace("\'", ' ')
df['text'] = df['text'].str.replace("]", ' ')
df['text'] = df['text'].str.replace("[", ' ')
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)

In [ ]:
df

In [ ]:
%%time
g=graphistry.nodes(df)
g2 = g.umap(df,**topic_model)
g2 = g2.dbscan() #min_dist=1, min_samples=3)
g3 = g2.transform_dbscan(df,return_graph=False)
df2=pd.DataFrame(g2.get_matrix())

max_index_per_row = df2.idxmax(axis=1)
top_3_indices = max_index_per_row.value_counts().index[:10]
top_3_indices

In [ ]:
g2.plot()

## featurize

In [ ]:
from DOTS.feat import featurize_stories
import graphistry
import umap

In [ ]:
featurize_stories(str(articles[711]), top_k = 3, max_len=512)

In [ ]:
rank_articles=[]
from tqdm import tqdm
indices3 = [i for i in range(len(articles)) if i not in indices2]

for i in tqdm(indices3):
    try:
        cc = featurize_stories(str(articles[i]), top_k = 3, max_len=512)
        rank_articles.append([cc])
    except:
        pass

In [ ]:
os.getcwd()

In [ ]:
flattened_list = [item for sublist in rank_articles for item in sublist]

In [ ]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key

In [ ]:
# pd.DataFrame(flattened_list).to_csv('big_test.txt')
# data=pd.read_csv('../../../big_test.txt',index_col=0)

In [ ]:
# import pandas as pd
data=pd.DataFrame(flattened_list)  # each ranked feature is a row
data=data.dropna(how='any')
data=data.drop_duplicates()

object_columns = data.select_dtypes(include=['object']).columns
data[object_columns] = data[object_columns].astype(str)

# g = graphistry.edges(data).materialize_nodes()
g = graphistry.nodes(data)
# g2 = g.featurize()
# g2._clustersummary()

g22 = g.umap()
g22=g22.dbscan()#min_dist=0.1, min_samples=3) #82 groups
g33 = g22.transform_dbscan(data,return_graph=False)
df22=pd.DataFrame(g22.get_matrix())

max_index_per_row = df22.idxmax(axis=1)
top_33_indices = max_index_per_row.value_counts().index[:10]
top_33_indices

In [ ]:
g22.plot() # .encode_point_color('_dbscan',palette=["hotpink", "dodgerblue"],as_continuous=True).plot()